# Voltage Control for the IEEE 14-Bus Test Case 
ELEC 571: TERM PROJECT

In this project, we use formulate and solve optimal control problems for the IEEE 14-bus system.  In particular, we compare two approaches:  
- **Model Predictive Control**, where the optimizaiton problem is solved online in a receding-horizon fashion, and
- **Data-Driven Predictive Control**, where the optimization problem is ___ .

OVERVIEW: 
1. Package imports 
2. Generate **simulation data** using ANDES for the IEEE 14-bus system with disturbances. Problem setup. This includes loading case file and disturbances. 
3. **System Identification** using SIPPY from the inputs and outputs of the system. 
4. **Verification** of estimated system ID parameters. 
5. MPC

### Package imports

In [5]:
#%% Package imports

# Install the Python control library
# %pip install -q control

# Numerical and CVXPY packages
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import andes
from sippy_unipi import system_identification as sysid
import casadi as ca 
from control import ss


# Plotting
import matplotlib.pyplot as plt
# %config InlineBackend.figure_format = 'svg'
plt.style.use('tableau-colorblind10')

In [ ]:
x = np.zeros((4, 1))
y = np.ones((4, 1))

idx = x==y
print(idx)

### Generate Dynamic Simulation Data

---

Load the IEEE 14-bus test case and run an open loop simulation to collect data. 
The case file (.xlsx) contains all model information as well as disturbances.

The system contains 14 buses, 5 synchronous generators, 11 loads. 
We are interested in voltage and frequency control. 

ANDES is an open-source Python library for dynamic simulation tool for power systems. 

The ultimate goal is to achieve transient stability. This means voltage and frequency control despite disturbances such as fault, load variation, etc. 
- **Inputs**: There are three controllable loads at buses X, Y, and Z, where the active power may be manipulated to achieve transient stability. 
- **Outputs**: We are interested in voltages at buses 2, 4, 7 and 9.

Simulation duration is 20 seconds with a time-step of $1/30 \approx 33.3$ ms. 

**Disturbances**: Persistent excitation via a 5-second chirp is applied for system identification. The chirp signal is of the form: 
$$u_t = P_t + A_P * \sin (0.2 \pi t^2)$$

where $A_P$ is the amplitude of the chirp signal and is given by 0.03 p.u., 0.02 p.u.and 0.02 p.u. for loads 1, 2 and 10, respectively. $P_t$ is the current active power of each load. The inputs are zero otherwise. $u_t^{(1)}$, $u_t^{(2)}$, and $u_t^{(3)}$, correspond to the active loads at buses 2, 3 and 13. See figure below for more details. 
$$ u_t = \begin{bmatrix}
u_t^{(1)} \\
u_t^{(2)} \\
u_t^{(3)} 
\end{bmatrix}
$$
![alt text](Figures/Single-line-diagram-of-the-IEEE-14-bus-test-system.png "Single-line-diagram-of-the-IEEE-14-bus-test-system")


In [6]:
# Simulation parameters
T_END = 20.0 # seconds
dt = 1/50 # time step = 0.020 seconds = 20.0 ms
K = round(T_END/dt + 1) # number of simulation steps (incl t = 0)
print(K)
# Outputs
p = 5 # Number of outputs; 4 voltages + 1 freq dev

# Inputs 
m = 3 # Number of inputs

# Disturbances
u = np.zeros((K, m))
for i in range(K):
    if i*dt <= 5:
        u[i, 0] = 0.03 * np.sin(2*np.pi * 0.5 * (i*dt)**2 /5)
    elif i*dt>=6 and i*dt <= 11: 
        u[i, 1] = 0.02 * np.sin(2*np.pi * 0.5 * (i*dt - 6)**2 /5)
    elif i*dt >=12 and i*dt <= 17:
        u[i, 2] = 0.02 * np.sin(2*np.pi * 0.5 * (i*dt - 12)**2 /5)

1001


In [7]:
#%% Generate data for system identification

# Load the system case
case = 'IEEE_cases/ieee14_alter_v2.xlsx'  

ssys = andes.load(case)

# Set load to constant active and reactive power (ANDES requirement)
ssys.PQ.config.p2p = 1.0
ssys.PQ.config.p2i = 0
ssys.PQ.config.p2z = 0

ssys.PQ.config.q2q = 1.0
ssys.PQ.config.q2i = 0
ssys.PQ.config.q2z = 0

# Run power flow simulation - initializes system
ssys.PFlow.run()

# Run dynamic simulation for 20 seconds
ssys.TDS.config.tf = T_END
ssys.TDS.config.tstep = dt
print('Running ANDES dynamic simulation...')
ssys.TDS.run()
print('Simulation finished')

# Plotting the results
ssys.TDS.load_plotter()
fig, ax = ssys.TDS.plt.plot(ssys.GENROU.omega)
plt.show()
fig.savefig("IEEE14_omega_plot.png", dpi=300)

fig, ax = ssys.TDS.plt.plot(ssys.GENROU.v)
fig.savefig("IEEE14_v_plot.png", dpi=300)



Running ANDES dynamic simulation...


  0%|          | 0/100 [00:00<?, ?%/s]

<Alter 2>: set PQ.PQ_1.Ppf.v=0.217021 at t=0.0333333. Previous value was 0.217.
<Alter 3>: set PQ.PQ_1.Ppf.v=0.217105 at t=0.0666667. Previous value was 0.217021.
<Alter 4>: set PQ.PQ_1.Ppf.v=0.217293 at t=0.1. Previous value was 0.217105.
<Alter 5>: set PQ.PQ_1.Ppf.v=0.217628 at t=0.133333. Previous value was 0.217293.
<Alter 6>: set PQ.PQ_1.Ppf.v=0.218152 at t=0.166667. Previous value was 0.217628.
<Alter 7>: set PQ.PQ_1.Ppf.v=0.218906 at t=0.2. Previous value was 0.218152.
<Alter 8>: set PQ.PQ_1.Ppf.v=0.219932 at t=0.233333. Previous value was 0.218906.
<Alter 9>: set PQ.PQ_1.Ppf.v=0.221272 at t=0.266667. Previous value was 0.219932.
<Alter 10>: set PQ.PQ_1.Ppf.v=0.222967 at t=0.3. Previous value was 0.221272.
<Alter 11>: set PQ.PQ_1.Ppf.v=0.22506 at t=0.333333. Previous value was 0.222967.
<Alter 12>: set PQ.PQ_1.Ppf.v=0.227591 at t=0.366667. Previous value was 0.22506.
<Alter 13>: set PQ.PQ_1.Ppf.v=0.230602 at t=0.4. Previous value was 0.227591.
<Alter 14>: set PQ.PQ_1.Ppf.v=0.234

/opt/anaconda3/envs/controls/lib/python3.13/site-packages/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/ft/6qjq8bkn2j1300glgtw50r8r0000gn/T/ipykernel_74752/1911045503.py:30: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/opt/anaconda3/envs/controls/lib/python3.13/site-packages/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
# Extract simulation (time-series) data 
data = ssys.dae.ts

# Extract variables of interest (time, frequency, voltages)
time = data.time
omega = data.x[:, ssys.GENROU.omega.a]
Vbus = data.y[:, ssys.Bus.v.a]


Time steps:  [0.00000e+00 1.00000e-04 2.01000e-02 ... 1.99601e+01 1.99801e+01
 2.00000e+01]


# System Identification
Model predictive control requires a state-space model. 
Consider the discrete-time linear time-invariant (LTI) system

$$
x_{k+1} = A\,x_k + B\,u_k,
$$
$$ 
y_k = C\, x_k + B\,u_k
$$

System identification is use to estimate the matrices $A, B, C$, and $D$. SIPPY, an open-source system identification package, is used to estimate matrices A, B, C, and D.The dynamic ANDES simulation data is used for system identification. We used N4SID as the method for system identification {cite}.



Note on model order: 


In [8]:
# 4) Subspace identification (N4SID) to obtain a reduced-order discrete-time model
# We'll identify a single MIMO model with inputs u (3) and outputs y (freq + voltages)

Y = df[['freq_dev'] + [f'V_b{b}' for b in buses_of_interest]].values
print('Output Y shape:', Y.shape)
U = df[[f'u{j+1}' for j in range(u.shape[1])]].values
print('Input U shape:', U.shape)
# Downsample data to reduce computation for identification (optional)


# Choose model order (experiment with this)
model_order = 17
print('Running N4SID (this may take a few seconds)...')
model = sysid(Y, U, 'N4SID', SS_fixed_order=model_order)

A_id = np.array(model.A)
B_id = np.array(model.B)
C_id = np.array(model.C)
D_id = np.array(model.D)
print('Identified model shapes: A', A_id.shape, 'B', B_id.shape, 'C', C_id.shape, 'D', D_id.shape)


# Convert identified model to discrete-time state-space in control package
sys_id = ss(A_id, B_id, C_id, D_id)
print('Identified discrete-time state-space model: \n', sys_id)

NameError: name 'df' is not defined